**Note:** A single simulation is performed with results being saved in an HDF file.

**Warning:** If a `results` directory exists in the working directory, old results will be lost.

In [1]:
%load_ext Cython
import time
import os
import shutil
from collections import namedtuple
import numpy as np
import h5py
import pickle

Created `%t` as an alias for `%timeit`.
Created `%%t` as an alias for `%%timeit`.


In [2]:
%%cython
#cython: wraparound=False, boundscheck=False, cdivision=True
#cython: profile=False, nonecheck=False, overflowcheck=False
#cython: cdivision_warnings=False, unraisable_tracebacks=False
import numpy as np
cimport numpy as np

cpdef iterate(Z, c):
    '''Element by elemenent iteration.

    Args:
        Z (ndarray - int) - Represents 2D space
        c (namedtuple) - Container for constants
    Returns:
        Z (ndarray - int)
    '''

    # Holds the sum of neighbors for each element in Z
    N = np.zeros((c.rows-1, c.cols-1), dtype=int)

    cdef int rows = c.rows
    cdef int cols = c.cols
    cdef long [:, :] N_ = N
    cdef long [:, :] Z_ = Z
    cdef int x, y

    with nogil:
        
        # Count neighbors
        for x in range(1, rows-1):
            for y in range(1, cols-1):
                N_[x, y] = (Z_[x-1, y-1] + Z_[x-1, y] + Z_[x-1, y+1] +
                            Z_[x,   y-1]              + Z_[x,   y+1] +
                            Z_[x+1, y-1] + Z_[x+1, y] + Z_[x+1, y+1])
                
        # Apply rules 
        for x in range(1, rows-1):
            for y in range(1, cols-1):
                if Z_[x, y] == 1 and (N_[x, y] < 2 or N_[x, y] > 3):
                    Z_[x, y] = 0
                elif Z_[x, y] == 0 and N_[x, y] == 3:
                     Z_[x, y] = 1

    return np.array(Z_)

In [3]:
def init_grid(c):
    '''Initialized the 2D grid with random values, with an empty border.'''

    Z = np.random.randint(0, 2, (c.rows, c.cols))
    
    Z[0, :] = 0
    Z[-1, :] = 0
    Z[:, 0] = 0
    Z[:, -1] = 0
    
    return Z

In [4]:
# Configure parameters
Const = namedtuple('c', ['rows', 'cols', 'n_iterations', 'rootdir'])
c = Const(rows=150, cols=150, n_iterations=600, rootdir='./results/')

# Create a fresh results directory
if os.path.exists(c.rootdir):
    shutil.rmtree(c.rootdir)
os.makedirs(c.rootdir)

pickle.dump(c._asdict(), open('./results/c.p', 'wb'))

In [5]:
start = time.time()

# Initialize grid
Z = init_grid(c)

Z_history = np.empty((c.rows, c.cols, c.n_iterations), dtype=int)
Z_history[:, :, 0] = Z  # Initial state

for i in range(1, c.n_iterations):
        
    Z = iterate(Z, c)
    
    Z_history[:, :, i] = Z 

print('Time elapsed: ', time.time() - start)

Time elapsed:  0.305417537689209


In [6]:
# Create new store
f = h5py.File(os.path.join(c.rootdir, 'results.hdf5'), 'w')
dset = f.create_dataset("Results", (c.rows, c.cols, c.n_iterations), dtype=int)
dset[...] = Z_history[...]
f.close()